# AEC Scripts

In [1]:
# common imports
from arcgis.gis import GIS

## User Input
#### GIS Configuration
For each GIS, enter portal url (such as `https://www.arcgis.com` for ArcGIS Online), username, and password

In [ ]:
# log in to respective portals
template = GIS("<template GIS>", "<un>", "<pw>")
project = GIS("<project GIS>", "<un>", "<pw>")
delivery = GIS("<delivery GIS>", "<un>", "<pw>")

#### Script Configuration
Defines how to clone the template organization

In [5]:
# path to csv with users to be added
USER_CSV = "<CSV PATH>"

# groups to be copied from template to project organization
PROJECT_GROUPS = ["data", "maps", "internal apps"]
# groups to be copied from template to delivery organization
DELIVERY_GROUPS = ["external apps"]

# new item name structure, {} replaced by old item name
NAME_STRUCTURE = "<PROJECT NAME> {}"
# user folder for new items
FOLDER = NAME_STRUCTURE.format("Content")

# delivery organization UX component locations
THUMBNAIL_PATH = "<THUMBNAIL PATH>"
FOOTER_PATH = "<FOOTER PATH>"
BANNER_PATH = "<BANNER PATH>"
BACKGROUND_PATH = "<BACKGROUND PATH"
# featured content group
FEATURED_GROUP = "data"

## Utility Functions
*NOTE If unable to import local files, __replace__ the following cell with the __[code here](https://github.com/ArcGIS/aec-scripts/blob/master/clone_utils.py)__

In [ ]:
from clone_utils import copy_group_and_items, add_users_csv

## Execution
#### Copy Groups and Items

In [ ]:
# get groups
template_project_groups = [template.groups.search(group)[0] for group in PROJECT_GROUPS]
template_delivery_groups = [template.groups.search(group)[0] for group in DELIVERY_GROUPS]

# update titles to reflect projects
def modify(copy):
    copy['title'] = NAME_STRUCTURE.format(copy['title'])
    return copy

#define function arguments
item_args = {
    "owner": project.users.me,
    "folder": FOLDER,
    "modify_items": modify
}
group_args = {"modify": modify}

# copy the groups
project_groups = [copy_group_and_items(template, group, project, group_args, item_args)
                  for group in template_project_groups]
delivery_groups = [copy_group_and_items(template, group, delivery, group_args, item_args)
                  for group in template_delivery_groups]

#### Add Users

In [ ]:
# add users to delivery org
add_users_csv(USER_CSV, delivery, groups=delivery_groups)

#### Customize Delivery Organization UI

In [ ]:
ux = delivery.admin.ux
ux.set_background(BACKGROUND_PATH)
ux.set_banner(BANNER_PATH)
ux.set_logo(THUMBNAIL_PATH)

featured_group = next(group for group in delivery_groups if group.title == FEATURED_GROUP)
ux.featured_content = {'group': featured_group}